In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import pandas as pd
import numpy as np
import requests
import os

In [3]:
ADDRESS = "http://192.168.1.11"
PORT = 8000

participant_preferences_url = "{}:{}/iris/retrieve-participant-preferences".format(ADDRESS, PORT)

In [4]:
participant_id = 0
debrief = 0

transparency = 1

prototype_type = "random"

In [5]:
json={
    'participant_id': participant_id
}

req = requests.post(participant_preferences_url, json=json)
if req.ok:
    queries =pd.DataFrame(req.json()["data"])
    queries
else:
    # Stop, there will be no debrief!
    pass

In [6]:
# Dataframe that will be used to upload question IDs for cluster prototypes to database.
clusters_sql = pd.DataFrame(columns=['question_id', 'cluster_id', 'query_id'])

In [7]:
# Names of survey dataframe columns.
survey_column_name = "Survey Name"
question_column_name = "Question"
type_column_name = "Type"
offered_answers_name = "Official Answers"

In [8]:
survey_csv = pd.DataFrame(columns=[survey_column_name, 
                                   question_column_name,
                                   type_column_name, 
                                   offered_answers_name])

In [9]:
survey_csv.loc[0, survey_column_name] = "Debrief, Mission {}, Participant {}".format(debrief, participant_id)
survey_csv.loc[1, survey_column_name] = "Order"
survey_csv.loc[2, survey_column_name] = str(debrief)
survey_csv.loc[3, survey_column_name] = "Last Edited By"
survey_csv.loc[4, survey_column_name] = "Python Script"
survey_csv.loc[5, survey_column_name] = "Who knows?"

In [10]:
def select_random_queries(queries):
        new_queries = queries.sample(frac=1).copy()
        new_queries = new_queries.drop_duplicates(subset=['cluster_id'], keep='last')
        new_queries = new_queries.reset_index()
        return new_queries

In [11]:
if prototype_type == "random":
    prototypes = select_random_queries(queries)
else:
    # Do a different prototype selection method.
    pass

ValueError: a must be greater than 0

In [ ]:
def _high_transparency():
    for index, query in queries.iterrows():

        clusters_sql.loc[index,"question_id"]=index
        clusters_sql.loc[index,"cluster_id"]=query["cluster_id"]
        clusters_sql.loc[index,"query_id"]=query["query_id"]
        survey_csv.loc[index,type_column_name]="Debrief"
        
        if query['level_of_autonomy'] is 0:
            survey_csv.loc[index,question_column_name]= (
                'I asked Husky {0} to stop and wait '
                'for your response to this {1} query with a {2}% confidence level. '
                'How would you like me to handle similar queries next time?').format(
                    query['robot_id'],
                    query['type'],
                    query['confidence'])
            survey_csv.loc[index,offered_answers_name]=(
                'Please ask a Husky to handle similar queries in the next mission|'
                'Please ask a Husky to stop and wait until I answer similar queries in the next mission')
        else:
            survey_csv.loc[index,question_column_name]=(
                'Husky {0} answered this {1} query, with a {2} confidence level'
                'while you were tending to another query.'
                'How would you like me to handle similar queries next time?').format(
                    query["robot_id"],
                    query['type'],
                    query['confidence'])

            survey_csv.loc[index,offered_answers_name]=(
                'Please ask a Husky to handle similar queries in the next mission|'
                'Please ask a Husky to stop and wait until I answer similar queries in the next mission')


In [ ]:
def _low_transparency():
    print ("in low transparency")
    for index, query in queries.iterrows():

        clusters_sql.loc[index,"question_id"]=index
        clusters_sql.loc[index,"cluster_id"]=query["cluster_id"]
        clusters_sql.loc[index,"query_id"]=query["query_id"]
        survey_csv.loc[index,type_column_name]="Debrief"
        
        if query['level_of_autonomy'] == 0:
            survey_csv.loc[index,question_column_name]= ('This {0} query was answered by you. '
                'How would you like me to handle similar queries next time?').format(query["type"])
            survey_csv.loc[index,offered_answers_name]=('Please ask a Husky to handle similar queries in the next mission|'
                'Please ask a Husky to stop and wait until I answer similar queries in the next mission')
        else:
            survey_csv.loc[index,question_column_name]= ('This {0} query was answered by Husky {1} '
                'while you were attending to another query. How would you like me to handle similar queries next time?').format(
                query["robot_id"],
                query['type'])

            survey_csv.loc[index,offered_answers_name]=('Please ask a Husky to handle similar queries in the next mission|' 
                'Please ask a Husky to stop and wait until I answer similar queries in the next mission')

In [ ]:
if transparency:
    _high_transparency()
else:
    _low_transparency()

In [ ]:
# Remove NaN elements for readability in CSV.
survey_csv = survey_csv.replace(np.nan, '', regex=True)

outname = 'debrief{0}.csv'.format(debrief)
outdir = 'debrief_surveys'

if not os.path.exists(outdir):
    os.mkdir(outdir)

path = os.path.join(outdir, outname) 

survey_csv.to_csv(path, index=False)

In [ ]:
def send_query(query, URL):
    iData = {
                'sql': query,
                'database': 'dbsurveys'
            }
    req = requests.post(url=URL, data=iData)
    if "200" in req:
        print(query)
    print(req)
    print(req.text)

In [1]:
[cluster for index, cluster in clusters_sql.iterrows()]

NameError: name 'clusters_sql' is not defined

In [ ]:
",".join()

In [79]:
def send_clusters():
    iURL = 'http://192.168.1.11:8000/sql-temp/update'
    sql_template = 'INSERT dbexperiment.cluster_question SET VALUE '
    sql_template_value = '({0},{1})'
    
    query = sql_template
    for index, cluster in clusters_sql.iterrows():
        #print (cluster)
        query += sql_template_value.format( 
            cluster['cluster_id'],
            cluster['query_id'])
    send_query(query, iURL)

In [80]:
send_clusters()

question_id      0
cluster_id      -1
query_id       643
Name: 0, dtype: object
<Response [200]>
{"response":true,"data":{"fieldCount":0,"affectedRows":1,"insertId":0,"serverStatus":2,"warningCount":0,"message":"(Rows matched: 1  Changed: 0  Warnings: 0","protocol41":true,"changedRows":0}}
question_id      1
cluster_id       0
query_id       679
Name: 1, dtype: object
<Response [200]>
{"response":true,"data":{"fieldCount":0,"affectedRows":1,"insertId":0,"serverStatus":2,"warningCount":0,"message":"(Rows matched: 1  Changed: 0  Warnings: 0","protocol41":true,"changedRows":0}}
question_id      2
cluster_id      -1
query_id       680
Name: 2, dtype: object
<Response [200]>
{"response":true,"data":{"fieldCount":0,"affectedRows":1,"insertId":0,"serverStatus":2,"warningCount":0,"message":"(Rows matched: 1  Changed: 0  Warnings: 0","protocol41":true,"changedRows":0}}
question_id      3
cluster_id      -1
query_id       700
Name: 3, dtype: object
<Response [200]>
{"response":true,"data":{"fiel